### Crea un mapa con los eventos registrados en el archivo por el sensor GLM.

In [1]:
# Instala las librerías necesarias
!pip install netCDF4
!pip install datetime
!pip install glob
!pip install basemap
!pip install pandas


[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob

[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importa las librerías necesarias
import os
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from netCDF4 import Dataset
import netCDF4 as nc
import glob
import pandas as pd
import datetime
import cv2

In [3]:
# Define rutas de interes
ruta_notebook = os.getcwd()
ruta_figuras = os.path.join(ruta_notebook, 'Salidas/')
ruta_dir_padre = os.path.dirname(ruta_notebook)
ruta_files = ruta_dir_padre + '/Descarga/Archivos'

In [4]:
# Limpia los archivos .png ya existentes en Salida
files_existentes = glob.glob(os.path.join(ruta_figuras, '*.png'))
for f in files_existentes:
    os.remove(f)

In [5]:
# Limpia los archivos .mp4 ya existentes en Salida
files_existentes = glob.glob(os.path.join(ruta_figuras, '*.mp4'))
for f in files_existentes:
    os.remove(f)

In [6]:
# Carga las coordenadas del punto de interes
localidad = 'Buenos Aires'
lat_loc = -34.61
lon_loc = -58.39

In [7]:
# Area de graficado
delta = 2.00
lat_n_area = lat_loc + delta
lat_s_area = lat_loc - delta
lon_w_area = lon_loc - delta
lon_e_area = lon_loc + delta

In [8]:
# Ubica todos los archivos en la carpeta de repositorio
files = glob.glob(ruta_files + '/*.nc')
for f in files:

    #Carga las variables desde los archivos
    filename = f
    barrido = filename[-49:-35]                      # Fecha y hora del escaneo (archivo)
    print(barrido)
    data = Dataset(filename)
    group_lat = data.variables['group_lat'][:]       # Latitud de grupo (grados norte) [º]
    group_lon = data.variables['group_lon'][:]       # Longitud de grupo (grados este) [º]

    # Grafica los grupos registrados
    fig = plt.figure(figsize=(8, 6), facecolor='black')

    m = Basemap(projection='merc',
                llcrnrlat=lat_s_area, urcrnrlat=lat_n_area,
                llcrnrlon=lon_w_area, urcrnrlon=lon_e_area,
                resolution='i')

    # Fondo y continentes
    m.drawmapboundary(fill_color='black')
    m.fillcontinents(color='dimgray', lake_color='black')

    # Costas, países, estados
    m.drawcoastlines(color='white')
    m.drawcountries(color='white')
    m.drawstates(color='white', linewidth=0.5, linestyle='dotted')

    # Puntos de detección
    m.scatter(group_lon, group_lat, latlon=True, s=10, c='red', alpha=1)

    # Localidad
    x_loc, y_loc = m(lon_loc + 0.06, lat_loc)
    plt.text(x_loc, y_loc, localidad, color='white', fontsize=10)
    plt.text(1.0, 1.0, 'Group detections', color='red', fontsize=10)

    # Guarda figura
    plt.savefig(os.path.join(ruta_figuras, f'Mapa_Group_{barrido}.png'),
                dpi=300, bbox_inches='tight', facecolor='black')
    
    # Cierra figura
    plt.close(fig)

    del filename, data, barrido, group_lat, group_lon

del files

20252552331000
20252552331200
20252552331400
20252552332000
20252552332200
20252552332400
20252552333000
20252552333200
20252552333400
20252552334000
20252552334200
20252552334400


### Animación de grupos

In [9]:
# Encuentra todas las imágenes PNG y ordénalas por nombre
imagenes = sorted(glob.glob(ruta_figuras + '*.png'))

In [10]:
# Se asegura que hay imágenes
if len(imagenes) == 0:
    raise ValueError("No se encontraron imágenes en la carpeta especificada.")

In [11]:
# Obtiene las dimensiones de la primera imagen
frame = cv2.imread(imagenes[0])
altura, ancho, canales = frame.shape

In [12]:
# Define el nombre y codec del video de salida
nombre_video = os.path.join(ruta_figuras, 'animacion_groups_GLM.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para mp4
fps = 1  # frames por segundo

In [13]:
# Crea el objeto VideoWriter
video = cv2.VideoWriter(nombre_video, fourcc, fps, (ancho, altura))

In [14]:
# Agrega cada imagen al video
for img_path in imagenes:
    frame = cv2.imread(img_path)
    video.write(frame)

In [15]:
# Libera el objeto VideoWriter
video.release()